In [1]:
import xrft
import numpy as np
import xarray as xr
from xmitgcm import open_mdsdataset
from matplotlib import pyplot as plt
%matplotlib inline
from xgcm import Grid
from matplotlib import colors, ticker, cm
from xhistogram.xarray import histogram 
import pandas as pd
from matplotlib import colors as cols

In [2]:
import matplotlib 
font = {'family' : 'sans-serif',
        'size'   : 12}

matplotlib.rc('font', **font)

In [3]:
npick = 24*1 # pick up npick file
dt_out = 90
T1 =  196560 # 3months + 1 day
T2months = 291060 # 4.5 months

iters = range(T1, T2months, dt_out*npick)
#iters = T2months

data_dir = '/swot/SUM04/dbalwada/channel_beta/01km/tracer_exp2/' 

ds = open_mdsdataset(data_dir, delta_t=40, iters= iters, 
                            prefix=['U','V','W','PTRACER01','tracer1_diag']
                     ,ignore_unknown_vars=True, geometry='cartesian')

In [4]:
horispace = 1e3
time_index = 0
depth = -50
f =  -0.83e-4
X = ds.XC/1e3
Y = ds.YC/1e3
daytime = 86400
boxarea = horispace**2
ybound = 5e5

In [5]:
grid = Grid(ds, periodic='X')

In [6]:
hab01L = xr.open_dataarray('/home/qx344/data/hab01-longtime')
hab05L = xr.open_dataarray('/home/qx344/data/hab05-longtime')
vortold = np.linspace(-7, 7, 240)
strainold = np.linspace(.5e-3, 7, 240)
binbox01old =  (vortold[1]-vortold[0])*(strainold[1]-strainold[0])

In [7]:
vort_1d = grid.interp(grid.interp((grid.diff(ds.V, 'X') - grid.diff(ds.U, 'Y', boundary='extend')),
                   'X'), 'Y', boundary='extend')/horispace

strain_1d = ((grid.diff(ds.U, 'X') - grid.diff(ds.V, 'Y', boundary='extend'))**2 + 
          (grid.interp(grid.interp(grid.diff(ds.V, 'X') + grid.diff(ds.U, 'Y', boundary='extend'),
                       'X'), 'Y', boundary='extend'))**2)**0.5/horispace

In [8]:
vort_1d = vort_1d.sel(YC = slice(ybound,2e6-ybound)).isel(Z=0)
strain_1d = strain_1d.sel(YC = slice(ybound,2e6-ybound)).isel(Z=0)

In [9]:
vort_1d.load();
strain_1d.load();

In [10]:
#### vs jpdf

vortband01 = np.linspace(-7, 7, 240)
strainband01 = np.linspace(.5e-3, 7, 240)

ybound = 5e5

binbox01 =  (vortband01[1]-vortband01[0])*(strainband01[1]-strainband01[0])

vort_1d = (vort_1d.sel(YC = slice(ybound,2e6-ybound))/f).rename('vort1_0')
strain_1d = (abs(strain_1d.sel(YC = slice(ybound,2e6-ybound))/f)).rename('strain1_0')


hist01 = histogram(vort_1d,strain_1d,dim = ['XC','YC'] , bins=[vortband01,strainband01])
hist01.load();


In [11]:
diffusion = grid.interp(ds.DFrITr01, 'Z', boundary='extend')

In [12]:
kppDif50 = diffusion.sel(Z=-50, method='nearest').sel(YC = slice(ybound,2e6-ybound)) 
kppDif50 = kppDif50.chunk({'XC': 2000, 'YC': 2000, 'time':40})
kppDif50.load()

hab01_Diff50 = histogram(vort_1d,strain_1d, weights = kppDif50, dim = ['XC','YC'] , 
                         bins=[vortband01,strainband01])
hab01_Diff50.load();

In [13]:
surfTracer = ds.PTRACER01

sorfForce = (1-surfTracer).isel(Z=0).sel(YC = slice(ybound,2e6-ybound))/(72*60)


hab01_1SF = histogram(vort_1d,strain_1d, weights = sorfForce, dim = ['XC','YC'] , bins=[vortband01,strainband01])
hab01_1SF.load();

In [14]:
hab01_Diff50_mean = hab01_Diff50.mean(dim='time')

In [15]:
hab01_1SF_mean = hab01_1SF.mean(dim='time')